
# Présentation

Ce code a été développé lors du hackathon Insee 2018 dans l'objectif de lire le fichier SIRUS. Ce fichier étant trop lourd pour tenir en mémoire, il nous a fallu ruser et créer un autre csv, avec uniquement les informations nous intéressant dans SIRUS (les informations concernantles APET des secteurs O, P et Q.

Description technique du problème : une colonne de SIRUS désigne les codes APET, et nous n'avions besoin que des lignes dont les APET commençaient par "84", "85", "86", "87", "88", "90", "91", "92", "93", "94", "95", "96", "97", "98" ou "99".
Résolution : on lit le fichier SIRUS par groupe de 100 000 lignes, on ne conserve de chaque groupe que les lignes ayant les bons APET, que l'on ajoute DataFrame, qui se remplit donc progressivement. A la fin, on dispose d'un DataFrame ayant l'information que l'on souhaite, et on l'exporte en csv.

Pistes pour une utilité plus large : en appliquant une autre fonction que "right_apet", on peut créer un sous-échantillon de n'importe quel fichier sur n'importe quelle condition sur n'importe quelle colonne. Le script qui suit est une ébauche pour la sélection d'information dans de csv, aussi gros soient-il.

In [ ]:
import pandas as pd
import os
import numpy as np
# chemin a modifier
os.chdir("C://Users//alize//OneDrive//Documents//ENSAE//cesure//hackathon_insee")
filepath = "./données/DonneesTransmisesHackathon/sirus_2017.csv"

In [ ]:
def right_apet(apet_to_inspect, list_right_apet):
    """
    Return a list of booleans with the index of APET starting with one of the right APET.
    """
    if isinstance(apet_to_inspect, str):
        return apet_to_inspect[:2] in list_right_apet

In [ ]:
# Premier regard au csv, pour en connaitre les colonnes
# et le delimiteur.
with open(filepath, 'r') as f:
    first_line = f.readline()
    print(first_line)

sirus_id;nic;ape;apet;eff_3112_et;eff_etp_et;eff_et_effet_daaaammjj;enseigne_et1;nom_comm_et;adr_et_loc_geo;adr_et_compl;adr_et_voie_num;adr_et_voie_repet;adr_et_voie_type;adr_et_voie_lib;adr_et_cedex;adr_et_distsp;sir_adr_et_com_lib;adr_et_post;adr_et_l1;adr_et_l2;adr_et_l3;adr_et_l4;adr_et_l5;adr_et_l6;adr_et_l7;nic_siege;unite_type;region;adr_depcom;region_impl;region_mult;tr_eff_etp;cj;denom;denom_condense;sigle;enseigne;eff_3112_uniteLegale;eff_etp_uniteLegale;eff_effet_daaaammjj_uniteLegale;x;y;SourceXYW;qual



# Répartition des groupes de lignes

In [ ]:
# On compte le nombre de lignes (execution un peu longue, tout le fichier doit
# etre lu).
# NB : il aurait ete possible d'eviter cette etape : lire le fichier k lignes
# par k lignes et accepter que le dernier groupe de lignes
# puisse avoir moins de k lignes.
num_lines = sum(1 for line in open(filepath))
all_rows = list(range(num_lines + 1))
print(num_lines)

8526231


In [ ]:
header_row = 0
end = num_lines + 1

# Lecture par groupe de lignes du fichier

In [ ]:
# Initialisation : on prepare le DataFrame vide que l'on va 
# progressivement remplir.
df_all_header = pd.read_csv(filepath, sep=";", header=0, nrows=10, encoding="latin-1")
df_to_keep = pd.DataFrame(columns = df_all_header.columns)

# On fixe les APET que l'on cherche.
list_right_apet = ["84", "85", "86", "87", "88", "90", "91", "92", "93", "94", "95", "96", "97", "98", "99"]

# On fixe le nombre d'elements par groupes (et donc le nombre de groupes).
size_group = 100000
nb_groups = int(end / size_group)
groups = {}

for i in range(nb_groups):
    # construction des index des lignes a ajouter a df_to_keep
    if i != nb_groups:
        groups[i] = list(range(size_group * i + 1, size_group * (i+1)))
        groups[i].append(header_row)
    else:
        groups[i] = list(range(size_group * i + 1, num_lines))
        groups[i].append(header_row)
    # on en deduit les index a ne pas conserver
    rows_to_exclude = list(set(all_rows).symmetric_difference(groups[i]))
    # on lit le fichier Sirus et on en extrait les lignes correspondant
    # aux APET qui nous interessent.
    df_all = pd.read_csv(filepath, sep=";", header=0, skiprows=rows_to_exclude, encoding="latin-1")
    df_all["apet"] = df_all["apet"].apply(lambda x: str(x))
    tmp_to_keep = df_all[df_all["apet"].apply(lambda x: right_apet(x, list_right_apet))]
    # on ajoute ces lignes a df_to_keep
    df_to_keep = df_to_keep.append(tmp_to_keep)
    print("Finished with group ", i)

# Observation du résultat

In [ ]:
df_to_keep["apet"].describe()